In [ ]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key=yymk81Bo0odf7J5MSnJ6rDrICbfCUjaQkfmVM7bc"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

In [ ]:
import json
data = json.loads(response.text)

In [ ]:
data

In [ ]:
categories = []

for i in data.get('competitions', []):
    category = i.get('category', {})  # Extract category details safely
    category_id = category.get('id')
    category_name = category.get('name')

    if category_id and category_name:  # Ensure valid data before appending
        categories.append({"category_id": category_id, "category_name": category_name})
print(categories)

In [ ]:
print(len(data['competitions']))
print(len(categories))

In [ ]:
unique_categories = {i['category_id']: i['category_name'] for i in categories}

unique_categories

In [ ]:
competitions = []

for i in data.get('competitions', []):  # Get 'competitions' safely
    competition_id = i.get('id')
    name = i.get('name')
    parent_id = i.get('parent_id')  # Might be missing
    comp_type = i.get('type')
    gender = i.get('gender')

    category = i.get('category', {})  # Safely get 'category' dictionary
    category_id = category.get('id')  # Get category ID safely

    if competition_id and name and category_id:  # Ensure required fields exist
        competitions.append([
            competition_id, 
            name, 
            parent_id, 
            comp_type, 
            gender, 
            category_id
        ])
competitions


In [ ]:
print(len(data['competitions']))
print(len(categories))
print(len(categories))

In [ ]:
import mysql.connector

# Establish connection to MySQL (without specifying database)
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="240996"
)

cursor = conn.cursor()

cursor.execute("DROP DATABASE Tennis_project")

# Step 1: Create Database (if not exists)
cursor.execute("CREATE DATABASE IF NOT EXISTS Tennis_project")

# Select the newly created database
cursor.execute("USE Tennis_project")

# Step 2: Create Categories Table
create_categories_table = """
CREATE TABLE IF NOT EXISTS Categories (
    category_id VARCHAR(50) PRIMARY KEY,
    category_name VARCHAR(100) NOT NULL
);
"""

# Step 3: Create Competitions Table
create_competitions_table = """
CREATE TABLE IF NOT EXISTS Competitions (
    competition_id VARCHAR(50) PRIMARY KEY,
    competition_name VARCHAR(100) NOT NULL,
    parent_id VARCHAR(50) NULL,
    type VARCHAR(20) NOT NULL,
    gender VARCHAR(10) NOT NULL,
    category_id VARCHAR(50),
    FOREIGN KEY (category_id) REFERENCES Categories(category_id) ON DELETE CASCADE
);
"""

# Execute Queries
cursor.execute(create_categories_table)
cursor.execute(create_competitions_table)

# Commit and close connection
conn.commit()
cursor.close()
conn.close()

print("Database and tables created successfully!")

In [ ]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",  # Replace with your MySQL username
    password="240996",  # Replace with your MySQL password
    database="Tennis_project"
    )

cursor = conn.cursor()

'''cursor.execute("truncate table Categories;")
cursor.execute("truncate table Competitions;")'''

# Insert Categories into Database (Avoid Duplicates)
for category_id, category_name in unique_categories.items():
    cursor.execute(
        "INSERT IGNORE INTO Categories (category_id, category_name) VALUES (%s, %s)",
        (category_id, category_name)
    )


for each_competition in competitions:
    cursor.execute('INSERT INTO Competitions (competition_id, competition_name, parent_id, type, gender, category_id) VALUES (%s, %s, %s, %s, %s, %s)', each_competition)

# Commit and close connection
conn.commit()
cursor.close()
conn.close()

print(" Data inserted successfully into Categories & Competitions tables!")